# 19 卷积层
## 从全连接到卷积
- 平移不变性
- 局部性
- 二维交叉相关/卷积
$\rightarrow$
<img src="https://s2.loli.net/2022/02/10/VpnHAMmKR2qxEar.png" width=70%>
- 对全连接层使用平移不变性和局部性得到卷积层
<img src="https://s2.loli.net/2022/02/10/jlrhCisBmXQWOM8.png" width=50%>

## 卷积层
- 边缘检测、锐化、高斯模糊
- 交叉相关vs卷积：由于对称性因此使用中没有区别
  - 一维与三维交叉相关
- 核矩阵和偏移：
  - 输入与和核矩阵进行交叉相关，加上偏移得到输出
  - 可学习

## 代码

In [ ]:
import torch
import torch.nn as nn

# 卷积函数定义:矩阵二维互相关运算
def conv2d(X, K):
    h, w = K.shape
    Y = torch.zeros([X.shape[0] - h + 1, X.shape[1] - w + 1]) # 这里不只是维度，更要有数值信息0
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()   # 居然漏了sum()，太可恶了
            
    return Y

# 类定义
class Conv2D(nn.Module):
    def _init_(self, kernel_size):
        super()._init_()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, X):
        return conv2d(X, self.weight) + self.bias

# 简单使用应用    
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

lr = 3e-2
for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y - Y_hat)**2     ## 梯度清空的操作对象是类
    conv2d.zero_grad()
    loss.sum().backward()       ## sum()
    # Y_hat = Y_hat - lr * conv2d.grad   ## [:]使用 # 我是脑残
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i + 1}, loss {loss.sum():.3f}')
    

# ps备注
# *是矩阵按位相乘

- 感受野越大越好？
  - 此思路与隐藏层的深度和宽度思想有关
- 损失函数曲线下降抖动剧烈？
  - batch_size或者lr增大

# 20 卷积层里的填充和步幅
## 填充和步幅
- 填充padding
  - 常用padding：$$p_h = k_h -1, p_w = k_w - 1$$
  - 输出维度：$$(n_h - k_h + 1 + p_h, n_w - k_w + 1 + p_w)$$

- 步幅stride
<img src="https://s2.loli.net/2022/02/10/HTBm1tQfDSb3rv4.png" width=70%>


In [ ]:
## 注意：padding参数传入是两侧都填充还是单侧填充
import torch
import torch.nn as nn

def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)    # 这东西可以增加维度
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape


- 超参数中重要程度：核大小 > 填充 > 步幅
- 卷积核边长一般选奇数
- 过拟合：验证集较好情况下可以很好避免过拟合


# 21 卷积层里的多输入多输出通道
## 多输入多输出通道
- 多个输入通道
   - 彩色图片RGB --> 灰度 丢失信息 
   - 输入$\textbf{X}$,核$\textbf{W}$,输出$\textbf{Y}$--通道累加输出
- 多输出通道
   - 输入$X:c_i×n_h×n_w$
   - 核$W:c_o×c_i×k_h×k_w$
   - 输出$Y: c_o×m_h×m_w$
- 多输入和多输出通道
   - 输出：识别特定模式
   - 输入：输入通道核识别并组合输入中的模式
- 1×1 卷积层
  - 不识别空间，只融合通道
  - 相当于输入形状为$n_hn_w × c_i$，权重为$c_0 × c_i$的全连接层
  
## 代码实现

In [ ]:
import torch
import d2l.torch as d2l

# 多通道输入
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))  

# 多输出
# 考虑物理意义，输出通道模式不同
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack((K, K + 1, K + 2))

# 1*1 卷积
def corr2d_multi_in_out_1multi1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1multi1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1, Y2).sum()) < 1e-6

## 简洁调用
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2) #第一二个参数

In [43]:
import torch
# 1.区别一下这些矩阵运算
K = torch.tensor([[[[1, 1, 1, 1]]]])
# K.reshape((1,) + K.shape)
# K = torch.stack((K, K + 1, K + 2), 0)
# print(K)

# 2.zip只会最多降低一维
X = torch.tensor([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]])
Y = torch.tensor([[2], [1], [5],[6]])

# for x, y in zip(X, Y):
   # print(f'{x} and {y}')
# 3.卷积主要是按位乘，区别matmul
# 4. cat与stack区别：续接与新增维度
Z = torch.tensor([[[5, 5, 5], [6, 6, 6]]])
print(torch.cat([X, Z], 0)) 

- 模型性能与计算性能
- 二维卷积 || 深度图--3D卷积

# 22 池化层
## 池化层
- 卷积层-位置敏感
 - 垂直边缘
 - 平移不变性
 
- 二维最大池化
  - 2 * 2最大池化可以容一像素移位
  - 填充，步幅和多个通道
     - 没有课学习参数
     - 每个输入通道应用池化层以获得相应的输出通道
     - 输出通道数=输入通道数
- 平均池化层
   - 最大：每个窗口最强模式信号

## 实现

In [2]:
import torch
import torch.nn as nn
import d2l.torch as d2l

# 实现池化层的正向传播
def pool2d(X, poolsize, mod='max'):
    h, w = poolsize.shape()
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mod == 'max':
                Y[i, j] = X[i:i + h, j:j + w].max()
            elif mod == 'avg':
                Y[i, j] = X[i:i + h, j:j + w].mean()             
    return Y
# 填充和步幅
# 注意参数padding=1是指两侧都添加了，因此是+2
X = torch.arange(16, dtype=torch.float32).reshape(1, 1, 4, 4)
pool2d = nn.MaxPool2d(3)   ##注意默认的padding与stride值
# pool2d(X)

pool2d = nn.MaxPool2d(3, padding=1, stride=2)

pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))

# 23 经典卷积神经网络LeNet
## LeNet
- 手写的数字识别
- MNIST
- 早期成功NN
- 总结：
  - 先使用卷积层学习图片空间信息
  - 然后使用全连接层来转换到类别空间
  
## 代码实现

In [12]:
import torch
import torch.nn as nn
import d2l.torch as d2l

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(-1, 1, 28, 28)  # 批量数，通道数，28*28
    
net = nn.Sequential(
    Reshape(),
    nn.Conv2d(1, 6, kernel_size=5, padding=2),
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), #窗口不重叠
    nn.Conv2d(6, 16, kernel_size=5),
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
net = net(X)
 for layer in net:
     #X = layer(X)
     print(layer._class_._name_, 'output: \t', X.shape) 
## (1,1,28,28)--(1,6,28,28)--(1,6,14,14) 
## --(1,16,10,10)--(1,16,5,5)
## --(16*5*5, 120)--(120, 84)--(84,10)

def evaluate_accuracy_gpu(net, data_iter, device=None):
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device
    
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            X = [x.to(device) for x in X]
        else
            X = X.to(device)
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]


# 在GPU上训练的修改：
net.to(device)
X, y = X.to(device), y.to(device)
test_acc = evaluate_accuracy_gpu(net, test_iter)


不断压缩，提升通道维度

# 24 深度卷积神经网络 AlexNet

# 27 含并行连结的网络 GoogLeNet/Inception V3

- 1 * 1 卷积核：改变通道数
- 卷积核：3 * 3, padding=1; 5 * 5, padding=2(两侧)

#### Inception块：
- concat: 维度不变，将四种都连接在一起
- 每条路上通道数不同，
   - 1 * 1 卷积，抽取信息与降低通道数
   - 64：128：32：32
- 更少的参数设计和复杂度提升

#### GoogLeNet
- stage：高宽减半
<img src="https://s2.loli.net/2022/02/13/zS2Ki6Imx3dl8Nb.png" width=70%>
- 通道分配，输出通道增加

#### Inception 有各种后续变种
- BN(v2) batch normalization
- V3 修改了Inception块
   - 替换卷积层
   - kernel修改
   - 更深
- V4 使用残差连接

#### 总结：
- 四条有不同超参数的卷积层和池化层的路来抽取不同信息

## QS
- 计算量
- 经典模型：
   - 修改通道数，降低计算量
   - 输入输出拉长/拉宽等修改
- 核尺度修改：3 * 3-->1 * 3 + 3 * 1 可以降低计算量
- DensNet与全连接相同/Flatten：保留通道数，其它展开
